In [104]:
import torch
import torch.nn as nn
class Mynetwork(nn.Module):
    def __init__(self,input_num=1 , out_num=1,hidden_num=32):
        super().__init__()
        self.MLP=nn.Sequential(
            nn.Linear(input_num, hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,hidden_num),
            nn.ELU(),
            nn.Linear(hidden_num,out_num),
        )
        pass
    def forward(self,x):
        return self.MLP(x)

In [105]:
A=torch.tensor([[3/2,-1/2],[-1/2,3/2]])

eigenvalues= torch.linalg.eigvalsh(A)
print(eigenvalues)

tensor([1., 2.])


In [106]:
dtype=torch.float32
NN=Mynetwork(1,1).to(dtype=dtype)
input=torch.tensor([[1.0],[2.0]],dtype=dtype)
print(NN(input))

tensor([[0.2741],
        [0.3247]], grad_fn=<AddmmBackward0>)


In [107]:
output=NN(input)
diag=torch.diag(output.flatten())
B=torch.tensor([[0,-1/2],[-1/2,0]],dtype=torch.float32)+diag
print(B)

tensor([[ 0.2741, -0.5000],
        [-0.5000,  0.3247]], grad_fn=<AddBackward0>)


In [108]:
pre_eig=torch.linalg.eigvalsh(B)
print(pre_eig)

tensor([-0.2012,  0.8000], grad_fn=<LinalgEighBackward0>)


In [109]:
optimizer=torch.optim.Adam(NN.parameters(),lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=20,threshold=1e-4)
loss_fn=nn.L1Loss()
for i in range(1000):
    optimizer.zero_grad()
    
    output=NN(input)
    diag=torch.diag(output.flatten())
    B=torch.tensor([[0,-1/2],[-1/2,0]],dtype=torch.float32)+diag
    pre_eig=torch.linalg.eigvalsh(B)
    
    loss=loss_fn(eigenvalues[0],pre_eig[0])
    loss.backward()
    # print(NN.MLP[0].weight.grad)
    optimizer.step()
    scheduler.step(loss)
    if i %10==0:
        print(pre_eig)
        print(loss.item())

tensor([-0.2012,  0.8000], grad_fn=<LinalgEighBackward0>)
1.2012443542480469
tensor([1.0500, 2.2083], grad_fn=<LinalgEighBackward0>)
0.04997670650482178
tensor([0.9667, 2.1170], grad_fn=<LinalgEighBackward0>)
0.03326481580734253
tensor([0.9771, 2.1287], grad_fn=<LinalgEighBackward0>)
0.022867143154144287
tensor([1.1202, 2.2926], grad_fn=<LinalgEighBackward0>)
0.1201789379119873
tensor([1.0487, 2.2060], grad_fn=<LinalgEighBackward0>)
0.0487445592880249
tensor([0.9945, 2.1430], grad_fn=<LinalgEighBackward0>)
0.005525171756744385
tensor([0.9811, 2.1274], grad_fn=<LinalgEighBackward0>)
0.018943488597869873
tensor([0.9810, 2.1272], grad_fn=<LinalgEighBackward0>)
0.019019722938537598
tensor([1.0131, 2.1641], grad_fn=<LinalgEighBackward0>)
0.013109803199768066
tensor([0.9953, 2.1435], grad_fn=<LinalgEighBackward0>)
0.004731297492980957
tensor([1.0028, 2.1522], grad_fn=<LinalgEighBackward0>)
0.002846240997314453
tensor([1.0012, 2.1504], grad_fn=<LinalgEighBackward0>)
0.0012464523315429688
tens